# Reinforcement Learning : Model Free RL

Python Reinforcement Learning : Chapter 04

Model Free RL : Monte Carlo Prediction & Control 

In [270]:
"""import libraries"""
import gym
import numpy as np
import re
from collections import defaultdict
from functools import partial
import matplotlib.pyplot as plt

In [28]:
# envList = list (gym.envs.registry.all() )
# envList = [re.sub("\)","", str(x).split("(")[1]) for x in envList]
# envList = sorted(envList)
# envList

In [183]:
"""create a simulation instance using make function"""
env = gym.make('Blackjack-v0')

In [195]:
"""initialize the environemnt using reset method"""
for i in range(10):
    print( env.reset() )

(11, 9, False)
(13, 8, False)
(8, 2, False)
(15, 3, False)
(19, 1, False)
(12, 5, False)
(20, 3, False)
(20, 10, False)
(15, 6, False)
(14, 10, False)


In [199]:
for i in range(10): 
    env.reset()
    print( env.step(np.random.choice([0,1])) )

((5, 6, False), 1.0, True, {})
((12, 7, False), -1.0, True, {})
((17, 3, False), -1.0, True, {})
((18, 10, False), 0.0, False, {})
((14, 8, False), -1.0, True, {})
((16, 7, False), -1.0, True, {})
((15, 10, False), 0.0, False, {})
((20, 1, False), 0.0, True, {})
((16, 7, False), 0.0, False, {})
((15, 1, False), 0.0, False, {})


In [187]:
#"""create the enviroment using render method"""
#"""returns a popup window display of the environment"""
#env.render( mode='human')

In [72]:
print('observation space:', env.observation_space )
print('state sample :', env.observation_space.sample() )

observation space: Tuple(Discrete(32), Discrete(11), Discrete(2))
state sample : (26, 6, 0)


In [48]:
print('action_space:', env.action_space )
print('action space  :', env.action_space.n)
print('action sample :', env.action_space.sample())

action_space: Discrete(2)
action space  : 2
action sample : 0


In [301]:
env.reset() 
print( env.dealer )
print( env.player )
print( env.reward_range )

[10, 3]
[5, 8]
(-inf, inf)


In [251]:
(env.observation_space.sample())

(1, 2, 0)

In [277]:
def sample_policy(observation):
    player_score, dealer_score, useable_ace = observation

    if player_score >= 20:
        return 0  # Stand
    else:
        return 1  # Hit

In [298]:
def generate_episode(env, policy):
    state_list, action_list, reward_list = [],[],[]
    
    observation = env.reset()
    
    while True:
        state_list.append(observation)
        action = sample_policy(observation)
        action_list.append(action)
        
        observation, reward, done, info = env.step(action)
        reward_list.append(reward)
        if done:
            break
            
    return state_list, action_list, reward_list

In [307]:
for i in range(1000):
    env.reset()
    x = generate_episode(env, 0)
    print( len( x[0]), end=',')

2,2,2,3,1,2,1,1,1,1,1,1,3,1,2,2,3,1,1,1,3,2,2,2,1,1,3,1,2,1,2,1,3,1,3,1,2,1,1,1,1,2,1,1,3,3,3,2,1,2,1,1,1,1,1,4,1,1,3,4,2,1,1,1,2,2,1,1,3,2,2,2,2,1,1,1,1,1,1,2,1,1,2,2,2,2,2,1,4,2,1,2,2,1,2,2,2,1,1,1,1,3,1,2,2,1,1,1,2,2,4,1,2,2,2,2,2,1,3,2,3,1,3,2,3,3,1,2,2,2,1,2,1,2,2,2,1,1,3,1,1,1,1,1,2,1,3,2,2,1,1,1,2,2,2,2,1,2,1,1,1,1,2,1,1,4,1,2,1,1,1,2,2,2,2,1,1,2,1,2,4,1,1,3,2,2,2,2,2,1,3,2,2,4,2,2,2,1,1,2,1,2,1,2,2,2,4,2,2,2,1,1,1,2,2,1,1,1,2,2,2,2,1,2,1,3,1,2,2,2,1,2,1,3,2,2,1,1,1,3,3,1,4,1,2,4,2,2,1,1,1,1,3,5,3,1,1,2,2,2,1,2,2,2,4,2,4,1,1,1,1,2,2,2,1,1,4,1,1,1,4,2,3,1,2,2,2,2,2,2,1,1,2,2,4,1,1,2,1,1,3,1,2,1,2,1,1,2,2,2,2,1,1,4,2,2,2,1,1,3,2,1,1,3,2,2,2,1,1,3,2,2,2,2,2,1,2,1,1,4,2,4,1,1,3,3,1,2,2,2,3,1,2,1,2,1,1,2,4,2,1,2,2,2,1,1,1,2,1,1,1,2,2,3,1,2,2,3,2,3,4,2,2,2,1,1,1,2,2,2,1,1,2,2,2,4,1,2,2,2,1,2,1,1,2,1,1,1,1,2,1,1,2,1,1,1,1,2,2,2,4,3,2,2,2,2,5,1,1,1,4,1,1,2,2,2,1,1,1,2,4,2,2,2,1,2,2,1,1,1,1,5,2,2,1,2,3,2,1,1,1,3,1,5,1,2,1,1,2,1,3,1,1,2,2,2,2,1,1,1,1,1,1,3,2,2,3,2,2,1,1,1,2,3,1,1,1,1,2,1,

In [310]:
def firstvisit_mc_prediction(env, policy, gamma=1.0, number_of_espisodes=10000):
    V = defaultdict(float)
    N = defaultdict(int) 
    
    for episode in range(number_of_espisodes):
        states, actions, rewards = generate_episode(env, policy)
        G = 0.0
        
        for t in range(len(states)-1, -1, -1):
            R = rewards[t]
            S = states[t]
            G = gamma*G + R
            
            if S not in states[:t]:
                N[S] += 1
                V[S] += (G - V[S])/N[S]
    
    return V, N

In [311]:
firstvisit_mc_prediction(env, 0, 1, 10000)

(defaultdict(float,
             {(18, 6, False): -0.6859504132231402,
              (21, 7, True): 0.8823529411764703,
              (21, 8, False): 0.9189189189189187,
              (19, 8, False): -0.6410256410256407,
              (18, 8, False): -0.6285714285714284,
              (12, 8, False): -0.430232558139535,
              (18, 9, False): -0.5739130434782608,
              (15, 9, False): -0.6517857142857141,
              (15, 9, True): -0.5833333333333333,
              (12, 9, True): -0.6,
              (20, 2, False): 0.6666666666666665,
              (15, 2, False): -0.6454545454545456,
              (5, 2, False): 0.11111111111111108,
              (15, 10, False): -0.7029972752043598,
              (19, 10, False): -0.7523148148148154,
              (17, 10, False): -0.7347417840375585,
              (21, 5, True): 0.8518518518518516,
              (14, 10, False): -0.6506666666666666,
              (13, 7, False): -0.4680851063829787,
              (21, 3, True): 0.9

In [92]:
# optimal_policy = extract_optimal_policy(value_table, gamma=1.0)
# print(optimal_policy)

In [223]:
gamma = 1.0
R = [1.,1.,1.,3.,1.]
retList = []

for time in range(5):
    Gk = 0.0
    for k in range(time,5):
        Gk += gamma**(k-time) * R[k]
    
    #print(time, Gk)
    retList.append(Gk)
print( retList )

[7.0, 6.0, 5.0, 4.0, 1.0]


In [224]:
gamma = 1.0
R = [1.,1.,1.,3.,1.]
retList = []

for time in range(5):
    Gk = 0.0
    for k in range(5-1,time-1,-1):
        Gk = Gk*gamma**k + R[k]
    #print(time, Gk)
    retList.append(Gk)
print( retList )

[7.0, 6.0, 5.0, 4.0, 1.0]


In [254]:
for i in range(5,-1,-1):
    print(i, end=',')

5,4,3,2,1,0,

In [13]:
env.close()